In [68]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
from nltk.tokenize import TreebankWordTokenizer
import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /Users/Pegah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
import sys
# Ensure that your kernel is using Python3
assert sys.version_info.major == 3

In [70]:
# import the json
import json
with open("updated_data.json", "r") as f:
    women_summaries = json.load(f)
    
# PEGAH (From A5)
num_women = len(women_summaries)
print("Loaded {} women".format(num_women))
print("Each woman's entry is a dictionary with the following keys...")
print(women_summaries[0].keys())

# print(len(women_summaries))
# print(women_summaries)
# print(women_summaries[0]['name'])
# print(women_summaries[0]['summary'])

Loaded 1378 women
Each woman's entry is a dictionary with the following keys...
dict_keys(['name', 'summary'])


### Assigning an index for each woman:

In [71]:
# PEGAH
# Here, we will assign an index for each woman. This index will help us access data in numpy matrices.
# woman_to_index = movie_id_to_index
woman_to_index = {woman:index for index, woman in enumerate([d["name"] for d in women_summaries])}


# and because it might be useful...
woman_name_to_index = {name:woman_to_index[name] for name in [d['name'] for d in women_summaries]}
woman_index_to_name = {v:k for k,v in woman_name_to_index.items()}

# print("The index of \"{}\" is {}".format(women_summaries[7]['name'], woman_name_to_index[women_summaries[7]['name']]))

# TF-IDF vectorizer

In [73]:
n_feats = 5000 # QUESTION: What does this do?
doc_by_vocab = np.empty([len(women_summaries), n_feats])

def build_vectorizer(max_features, stop_words, max_df=0.8, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object
    
    Params: {max_features: Integer,
             max_df: Float,
             min_df: Float,
             norm: String,
             stop_words: String}
    Returns: TfidfVectorizer
    """
    # YOUR CODE HERE
    return TfidfVectorizer(stop_words=stop_words, max_features=max_features, max_df=max_df, min_df=min_df, norm=norm)
    raise NotImplementedError()
    
tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([d['summary'] for d in women_summaries]).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

# Cosine similarity

In [36]:
def get_sim(wom1, wom2, input_doc_mat, woman_name_to_index):
    """Returns a float giving the cosine similarity of 
       the two movie transcripts.
    
    Params: {mov1: String,
             mov2: String,
             input_doc_mat: Numpy Array,
             movie_name_to_index: Dict}
    Returns: Float (Cosine similarity of the two movie transcripts.)
    """
    # YOUR CODE HERE
    
    a = input_doc_mat[woman_name_to_index[wom1]] #this gives us the [......] freq of all words
    b = input_doc_mat[woman_name_to_index[wom2]] 
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [37]:
"""Check that get_sim returns the correct output"""
print("Similarity: Mary Anning vs. Lilias Armstrong")
print("======")
test1 = get_sim('Lilias Armstrong', 'Mary Anning', doc_by_vocab, woman_name_to_index)
print(test1)
assert test1 < 1 and test1 > 0
print("")

# print("Similarity: Star Wars vs. Star Trek: Generations")
# print("======")
# test2 = get_sim('star wars', 'star trek: generations', doc_by_vocab, movie_name_to_index)
# print(test2)
# assert test2 < 0.25 and test2 > 0.20

Similarity: Mary Anning vs. Lilias Armstrong
0.14088923737130635



In [66]:
def build_woman_sims_cos(n_wom, woman_index_to_name,input_doc_mat,woman_name_to_index,input_get_sim_method):
    """Returns a movie_sims matrix of size (num_movies,num_movies) where for (i,j):
        [i,j] should be the cosine similarity between the movie with index i and the movie with index j
        
    Note: What you set the diagonal to doesn't really matter, 
    but you should set it to 1 to indicate that all movies are trivially perfectly similar to themselves.
    
    Params: {n_mov: Integer,
             movie_index_to_name: String,
             input_doc_mat: Numpy Array,
             movie_name_to_index: Dict,
             input_get_sim_method: Function}
    Returns: Numpy Array 
    """
    
    woman_sims = np.ones((n_wom, n_wom)) #instantiate array
    for i in range(0, n_wom):
        for j in range(0, n_wom):
            woman_sims[i,j] = input_get_sim_method(woman_index_to_name[i], woman_index_to_name[j], input_doc_mat, woman_name_to_index)
    return woman_sims
    raise NotImplementedError()

In [67]:
woman_sims_cos = build_woman_sims_cos(num_women, woman_index_to_name, doc_by_vocab, woman_name_to_index, get_sim)

KeyError: 6